## Deliverable 2. Create a Customer Travel Destinations Map.

In [64]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

In [65]:
# Import API key
from config import g_key

In [66]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [69]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../World_Weather_Analysis/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Castro,CL,2019-08-26 17:25:49,-42.48,-73.76,48.20,61,40,14.99,scattered clouds,0.000,0
1,1,Lebu,ET,2019-08-26 17:25:49,8.96,38.73,58.69,83,72,1.45,light rain,2.187,0
2,2,Sitka,US,2019-08-26 17:25:49,37.17,-99.65,90.00,46,6,21.00,clear sky,0.000,0
3,3,Torbay,CA,2019-08-26 17:25:49,47.66,-52.73,75.00,43,20,10.29,few clouds,0.000,0
4,4,Grandview,US,2019-08-26 17:25:50,38.89,-94.53,69.80,93,40,4.70,heavy intensity rain,0.000,0


In [68]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 30


In [70]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)


In [71]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                     0
City                        0
Country                     0
Date                        0
Lat                         0
Lng                         0
Max Temp                    0
Humidity                    0
Cloudiness                  0
Wind Speed                  0
Current Description         0
Rain inches (last 3 hrs)    0
Snow inches (last 3 hrs)    0
dtype: int64

In [72]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)


In [73]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [74]:
# 6a. Set parameters to search for hotels with 5000 meters.
hotels = []
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i in range(len(hotel_df)):
    
    # 6c. Get latitude and longitude from DataFrame
    lat = hotel_df.iloc[i]['Lat']
    lng = hotel_df.iloc[i]['Lng']
    
    params = {"location":f'{lat},{lng}',
              "radius": 5000, 
              "types": "lodging", 
             "key": g_key}
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotels.append(response['results'][0]['name'])
    except:
        hotels.append("")
hotel_df["Hotel Name"] = hotels

hotel_df
        

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [75]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [76]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))